# Extração de dados da Mátricula do Imóvel

O seguinte algoritmo tem como objetivo extrair os dados de uma matrícula de imóvel organizada pelo Cartório Eunápio Torres, a fim de trazer informações mais simplicadas do documento.

In [ ]:
# !pip install pdfplumber

import pdfplumber 
import pathlib
import json
import re
import os

## Funções Auxiliares de Extração

### Função para extrair o texto do PDF

Aqui, extraimos o texto de dentro do PDF, usamos uma função ```crop``` para fazer o recorte do arquivo, evitando a leitura de possíveis textos contidos nas margens.

In [283]:
def extrair_texto(caminho_pdf: str) -> str:
    """
    Extrai o texto do corpo principal de um PDF, ignorando as margens
    através da bounding box.
    """
    texto_completo = ""
    with pdfplumber.open(caminho_pdf) as pdf:
        for pagina in pdf.pages:
            # Dimensões da página
            largura = pagina.width
            altura = pagina.height
            
            # x0:  margem esquerda; top: margem superior; x1: margem direita; bottom: margem inferior
            bounding_box = (0.1 * largura, 0.25 * altura, 0.9 * largura, 0.9 * altura)
            
            # recorte da página no espaço definido
            pagina_recortada = pagina.crop(bounding_box)
            
            texto_pagina = pagina_recortada.extract_text()
            
            if texto_pagina:
                texto_completo += texto_pagina + "\n"
            
    # normalização de espaços
    texto = re.sub(r'\s+', ' ', texto_completo)
    blocos = re.sub(r'[Dd]ou f[eé].', '\n', texto).split('\n')

    return [b.strip() for b in blocos if b.strip()]

### Função para extrair o detalhamento do imóvel

Aqui, temos uma função responável por identificar todos o nome de todas as áreas que compõem o imóvel.

In [282]:
def extrair_composicao(descricao: str) -> str:
    """
    Extrai a descrição da composição do imóvel, lidando com a ordem
    variável das informações de área e comodidades.
    """
    composicao_texto = ""
    
    # "contendo" é o que indica o que vem primeiro
    match_contendo = re.search(r'contendo\s+(.*?)(?:\s*PROPRIETÁRIO|\s*TÍTULO ANTERIOR:)', descricao, re.IGNORECASE | re.DOTALL)
    
    # Padrao A:  áreas depois composição 
    if match_contendo:
        # Se encontrou "contendo", a composição é o que vem depois
        composicao_texto = match_contendo.group(1)
    else:
        # Padrão B: composição depois áreas
        # Captura tudo desde até encontrar "área" | fim do bloco
        match_padrao_a = re.search(r'^(.*?)(?:\s+com\s+área|\s*PROPRIETÁRIO)', descricao, re.IGNORECASE | re.DOTALL)
        if match_padrao_a:
            composicao_texto = match_padrao_a.group(1)
            
    return " ".join(composicao_texto.strip().split())

### Função para Identificar o resposável que construiu o imóvel

Nela, encontramos tanto o nome da empresa, quanto seu código de identificação (CPF, CNPJ, CGC).

In [ ]:
def extrair_proprietario_inicial(descricao: str) -> dict:
    """
    Extrai de forma estruturada as informações do proprietário inicial
    do imóvel a partir do bloco de descrição.
    """
    proprietario_dados = {}
    
    bloco_prop_match = re.search(r'PROPRIET[ÁA]RIO:\s*(.*?)\s*T[IÍ]TULO ANTERIOR:', descricao, re.DOTALL | re.IGNORECASE)
    
    if not bloco_prop_match:
        return None 
    
    texto_prop = bloco_prop_match.group(1)
    
    # nome do proprietário (até a primeira ", ")
    nome_match = re.search(r'^(.*?)(?:,\s)', texto_prop)
    if nome_match:
        proprietario_dados["nome"] = nome_match.group(1).strip().rstrip(',')
    
    # tipo e o número do identificador (CNPJ/CGC/CPF)
    id_match = re.search(r'inscrita no\s+([A-Z\/]+)[\s\S]*?(\d[\d\.\-\/]+)', texto_prop, re.IGNORECASE)
    if id_match:
        proprietario_dados["identificador"] = f"{id_match.group(1).strip()} {id_match.group(2).strip()}" 
        
    return proprietario_dados if proprietario_dados else None

### Função para Identificar o valor de Avaliação 

Nesta função, extraímos o valor da última avaliação feita ao imóvel, assim como a data próxima a este evento.

In [377]:
def extrair_avaliacao(bloco: str) -> dict:
    """
    Extrai a avaliação do imóvel e a data da avaliação.
    """
    avaliacao = {}

    data_match = re.search(r'datado de (\d{1,2} de [a-zA-Z]+ de \d{4}|\d{1,2}.*?\d{4})', bloco, re.IGNORECASE)
    
    if data_match: avaliacao["data"] = data_match.group(1).strip()

    # encontra o valor do imóvel, que está logo antes do ITBI
    valor_match = re.search(r'R\$ ?([\d\.]+,\d{2}).*?sendo (?:pago|recolhido) o ITBI', bloco, re.IGNORECASE)
    if valor_match: avaliacao["valor"] = "R$ " + valor_match.group(1).strip()

    return avaliacao if avaliacao else None

## Função que Descreve o imóvel por completo

Nesta função, usamos todas as funções auxiliares de extração, bem como resoluções internas para que possa ser feito o máximo de detalhamento das características do imóvel.

In [ ]:

def parse_descricao(bloco: str) -> dict:
    """Extrai dados principais do imóvel e proprietário inicial."""

    dados = {}

# ======================================================================

    # matrícula
    matricula = re.search(r'Matr[ií]cula\s*(\d{1,3}\.?\d{3})', bloco, re.IGNORECASE)
    if matricula: 
        dados["matricula"] = matricula.group(1)


    # cartório
    cartorio = re.search(r'Registro Geral do (.*?)\)', bloco, re.IGNORECASE)
    if cartorio: dados["cartorio"] = cartorio.group(1)


    # número de Ordem e data de abertura
    ordem = re.search(r'ordem\s*(\d{1,3}\.?\d{3}).*?data de\s*(\d{1,2}.*?\d{4})', bloco, re.IGNORECASE)
    if ordem:
        dados["ordem"] = ordem.group(1)
        dados["data_abertura"] = ordem.group(2)
        

# ======================================================================
    # Transcrição
    padrao_geral = r'Transcri[cç][aã]o:\s*(.*?),\s*situado\s*[aàá]?\s*(.*?),\s*nesta cidade'
    match_geral = re.search(padrao_geral, bloco, re.IGNORECASE | re.DOTALL)

    if match_geral:
        dados.setdefault("imovel", {})
        tipo_descricao = match_geral.group(1).strip()
        dados["imovel"]["tipo"] = " ".join(tipo_descricao.split())
        
        endereco = match_geral.group(2).strip()
        dados["imovel"]["endereco"] = " ".join(endereco.split())

# ======================================================================
    # Áreas
    padroes_area = [
        ("privativa", r'(?:privativa real|real privativa).*?([\d\.,]+)'),
        ("uso_comum", r'(?:uso comum real|real de uso comum).*?([\d\.,]+)'),
        ("total", r'real total (?:de)?\s*([\d\.,]+)'),
        ("privativa_acessoria", r'privativa acess[oó]ria.*?([\d\.,]+)'),
        ("privativa_total", r'privativa total\s*([\d\.,]+)'),
        ("equiv_constr_tt", r'constru[cç][aã]o total (?:de)?\s*([\d\.,]+)'),
        ("fração_ideal", r'fra[cç][aã]o ideal (?:de)?\s*([\d\.,]+)'),
        ("coef_proporcionalidade", r'coeficiente de proporcionalidade\s*([\d\.,]+)'),
        ("cota_terreno", r'cota ideal do terreno (?:de)?\s*([\d\.,]+)')
    ]

    # composição do imóvel
    desc_geral_match = re.search(r'compost[ao]\s+de:(.*?)(?:PROPRIETÁRIO:)', bloco, re.IGNORECASE | re.DOTALL)
    
    if desc_geral_match:
        bloco_descricao_geral = desc_geral_match.group(1)
        areas = {}
        # Busca de todas as áreas
        for chave, area in padroes_area:
            match = re.search(area, bloco_descricao_geral, re.IGNORECASE)
            if match:
                valor_numerico = float(match.group(1).replace('.', '').replace(',', '.'))
                areas[chave] = valor_numerico

        if areas: dados["imovel"]["areas"] = areas

        # Busca da composição do imóvel
        composicao = extrair_composicao(bloco_descricao_geral)

        if composicao: dados["imovel"]["composicao"] = composicao
# ======================================================================

    # Proprietário inicial
    proprietario = extrair_proprietario_inicial(bloco)
    if proprietario:
        dados["proprietario_inicial"] = proprietario
        
    return dados

## Função para resumir toda a Descrição

In [401]:
def gerar_resumo_descricao(dados: dict) -> str:
    """
    Gera um texto resumo legível a partir do JSON criado pelo parse_descricao.
    """
    partes = []

    # Matrícula e cartório
    if "matricula" in dados:
        partes.append(f"Matrícula nº {dados['matricula']}")
    if "cartorio" in dados:
        partes.append(f"do {dados['cartorio']}")
    if "ordem" in dados and "data_abertura" in dados:
        partes.append(f"(Ordem {dados['ordem']}, aberta em {dados['data_abertura']}).")
    else:
        partes.append(".")

    # Imóvel
    imovel = dados.get("imovel", {})
    if imovel:
        tipo = imovel.get("tipo")
        endereco = imovel.get("endereco")
        if tipo and endereco:
            partes.append(f"Trata-se de {tipo.lower()}, localizado em {endereco}.")
        elif endereco:
            partes.append(f"Imóvel localizado em {endereco}.")

        # Áreas
        if "areas" in imovel:
            areas = imovel["areas"]
            descr_areas = []
            if "privativa" in areas:
                descr_areas.append(f"área privativa de {areas['privativa']} m²")
            if "uso_comum" in areas:
                descr_areas.append(f"área de uso comum de {areas['uso_comum']} m²")
            if "total" in areas:
                descr_areas.append(f"área total de {areas['total']} m²")
            if "fração_ideal" in areas:
                descr_areas.append(f"fração ideal de {areas['fração_ideal']}")
            if "cota_terreno" in areas:
                descr_areas.append(f"cota ideal do terreno de {areas['cota_terreno']} m²")

            if descr_areas:
                partes.append("Possui " + ", ".join(descr_areas) + ".")

        # Composição
        if "composicao" in imovel and imovel["composicao"]:
            partes.append("O imóvel é composto por " + imovel["composicao"] + ".")

    # Proprietário inicial
    prop = dados.get("proprietario_inicial")
    if prop:
        nome = prop.get("nome")
        cnpj = prop.get("cnpj")
        representante = prop.get("representante")
        if nome:
            resumo_prop = f"Proprietário inicial: {nome}"
            if cnpj:
                resumo_prop += f", inscrito no CNPJ {cnpj}"
            if representante:
                resumo_prop += f", representado por {representante}"
            resumo_prop += "."
            partes.append(resumo_prop)

    # Valor de avaliação (se existir no nível raiz ou em avaliações)
    valor_avaliacao = dados.get("valor_avaliacao")
    data_avaliacao = dados.get("data_avaliacao")

    if not valor_avaliacao and "avaliacoes" in dados:
        for dado in dados["avaliacoes"]:
            if "valor" in dado:
                valor_avaliacao = dado["valor"]
                data_avaliacao = dado.get("data")
                break

    if valor_avaliacao:
        if data_avaliacao:
            partes.append(f"O imóvel teve uma avaliação registrada em {data_avaliacao}, no valor de R$ {valor_avaliacao}.")
        else:
            partes.append(f"O imóvel possui uma avaliação registrada no valor de R$ {valor_avaliacao}.")

    return " ".join(partes)


## Processamento de Arquivos de Entrada

Aqui, realizamos a leitura e síntese de todos os documentos presentes na pasta de entrada.

Os arquivos resultantes são armazenados na pasta ```resultados_json```

In [403]:
# --- BLOCO DE EXECUÇÃO PRINCIPAL (MODIFICADO) ---
if __name__ == "__main__":

    arquivos_pdf = [
        "2455251",
        "1444408441014",
        "1444404404084",
        "1444411417161",
        "1600000131788"
    ]

    pasta_name = "resultados_json"
    
    diretorio_atual = pathlib.Path.cwd()

    parent_dir = diretorio_atual.parent 

    # caminho diretório pai/ nova pasta
    pasta_resultados = parent_dir / pasta_name
   
    # criação da pasta
    pathlib.Path(pasta_resultados).mkdir(exist_ok=True)

    for nome_arquivo_pdf in arquivos_pdf:
        print(f"--- Processando Arquivo: {nome_arquivo_pdf}.pdf ---")
        
        # extração e segmentação do texto
        blocos = extrair_texto(f"../input/{nome_arquivo_pdf}.pdf")
        
        # descrição e busca de preço final do imóvel no documento
        if blocos:
            dados_estruturados = parse_descricao(blocos[0])
    
            for bloco in blocos[1:]:
                if re.search(r'CONSOLIDA[CÇ][AÃ]O DA PROPRIEDADE', bloco):
                    avaliacao = extrair_avaliacao(bloco)
                    if avaliacao:
                        dados_estruturados.setdefault("avaliacoes", []).append(avaliacao)
                        break

        # Etapa 3: Gerar o resumo (opcional, mas bom ter)
        resumo = gerar_resumo_descricao(dados_estruturados)
        dados_estruturados["resumo"] = resumo
        print("\n>>> RESUMO DA MATRÍCULA <<<\n")
        print(resumo)
        
        # salva o dicionário como arquivo JSON
        nome_arquivo_json = f"Resumo_{nome_arquivo_pdf}.json"
        caminho_saida = os.path.join(pasta_resultados, nome_arquivo_json)
        print(f"Salvando dados estruturados em: /{pasta_name}/{nome_arquivo_json}")

        # Abre o arquivo em modo de escrita ('w') com codificação 'utf-8'
        with open(caminho_saida, 'w', encoding='utf-8') as f:
            # Usa json.dump() para escrever o dicionário no arquivo
            json.dump(
                dados_estruturados, 
                f, 
                ensure_ascii=False, 
                indent=4
            )
        
        print("Arquivo JSON gerado com sucesso!")
        print("\n" + "="*50 + "\n")

--- Processando Arquivo: 2455251.pdf ---

>>> RESUMO DA MATRÍCULA <<<

Matrícula nº 85.648 do 2º Ofício do Registro de Imóveis Zona Norte (Ordem 85.648, aberta em 05 de outubro de 2009). Trata-se de unidade autonoma restaurante n.º 506, do edifício manaíra palace residence, localizado em Avenida João Mauricio, nº 581, esquina com a Avenida, Bananeiras, no Bairro de Manaíra. Possui área privativa de 177.8 m², área de uso comum de 82.88 m², área total de 260.68 m², fração ideal de 2.775, cota ideal do terreno de 51.07 m². O imóvel é composto por Restaurante, câmara frigorífica, sala de lavagem, duas salas de preparo e sala de cocção,. Proprietário inicial: Firma SAGRES CONSTRUTORA E EMPREENDIMENTOS IMOBILIÁRIOS LTDA.. O imóvel teve uma avaliação registrada em 24 de Abril de 2025, no valor de R$ R$ 1.807.000,00.
Salvando dados estruturados em: /resultados_json/Resumo_2455251.json
Arquivo JSON gerado com sucesso!


--- Processando Arquivo: 1444408441014.pdf ---

>>> RESUMO DA MATRÍCULA <<<

## Função de controle

Usada para realizar testes manuais, sem a necessidade de gerar um arquivo.

In [390]:
def teste_estrutural(caminho_pdf: str) -> dict:
    blocos = extrair_texto(caminho_pdf)
    estrutura = parse_descricao(blocos[0])

    for bloco in blocos[1:]:
        if re.search(r'CONSOLIDA[CÇ][AÃ]O DA PROPRIEDADE', bloco):
            avaliacao = extrair_avaliacao(bloco)
            if avaliacao:
                estrutura.setdefault("avaliacoes", []).append(avaliacao)
            break
    
    return estrutura

# Exemplo de uso:
arquivo = "../input/2455251.pdf"
arquivo = "../input/1444408441014.pdf"
arquivo = "../input/1444404404084.pdf"
arquivo = "../input/1444411417161.pdf"
arquivo = "../input/1600000131788.pdf"
resultado = teste_estrutural(arquivo)
print(json.dumps(resultado, indent=2, ensure_ascii=False))

{
  "matricula": "87.950",
  "cartorio": "2º Ofício do Registro de Imóveis Zona Norte",
  "ordem": "87.950",
  "data_abertura": "26 de março de 2010",
  "imovel": {
    "tipo": "APARTAMENTO n.º 1802, do EDIFICIO MANOÁ RESIDENCE",
    "endereco": "Rua Bacharel José de Oliveira Curchatuz, nº 527, esquina com a Rua Vanja Viana Sales, no bairro Aeroclube",
    "areas": {
      "privativa": 234.84,
      "uso_comum": 129.11,
      "total": 363.95,
      "equiv_constr_tt": 280.2,
      "fração_ideal": 2.18,
      "cota_terreno": 64.61
    },
    "composicao": "Varanda, sala de estar, sala de jantar, estar intimo, lavabo, quatro suítes sendo uma máster com varanda, cozinha, despensa, área de serviço, dependência de empregada e quatro vagas de garagem, sendo duas cobertas e duas descobertas,"
  },
  "proprietario_inicial": {
    "nome": "Firma CONSTRUTORA MASHIA LTDA.",
    "identificador_tipo": "CGC 04.534.882/0001-26"
  },
  "avaliacoes": [
    {
      "data": "10 de fevereiro de 2025",
    